In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import gym

%matplotlib inline

In [2]:
import tf_util
import load_policy

In [3]:
# tf_util.initialize()

In [4]:
sess = tf.InteractiveSession()

# with tf.Session():
#     tf_util.initialize()

In [5]:
policy_fn = load_policy.load_policy('./experts/Hopper-v1.pkl')

obs (1, 11) (1, 11)


observation is an 11-dim array

action is a (1, 3) shaped array

In [6]:
# None: batch size could vary
x_plh = tf.placeholder(tf.float32, shape=[None, 11])
y_plh = tf.placeholder(tf.float32, shape=[None, 3])

In [7]:
W_var = tf.Variable(tf.zeros([11, 3]))
b_var = tf.Variable(tf.zeros([3]))

In [8]:
sess.run(tf.global_variables_initializer())

In [9]:
y = tf.matmul(x_plh, W_var) + b_var

In [10]:
# cross_entropy = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits(labels=y_plh, logits=y))

loss_mse = tf.losses.mean_squared_error(labels=y_plh, predictions=y)

In [13]:
# train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss_mse)

In [14]:
train_step

<tf.Operation 'GradientDescent_1' type=NoOp>

In [15]:
obs, action = None, None
env = gym.make('Hopper-v1')
for _ in range(1000):
    if obs is None and action is None:
        obs = env.reset()
        action = policy_fn(obs[None,:])
    else:
        obs, r, done, _ = env.step(action)
        action = policy_fn(obs[None, :])
    _x = np.array([obs])
    _y = np.array([action.ravel()])
#     batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x_plh: _x, y_plh: _y})

In [16]:
# collect (obs, action) pairs for testing
obs_test, actions_test = [], []
for _ in range(100):
    obs, r, done, _ = env.step(action)
    action = policy_fn(obs[None, :])
    obs_test.append(obs)
    actions_test.append(action.ravel())
obs_test = np.array(obs_test)
actions_test = np.array(actions_test)

In [17]:
obs_test.shape

(100, 11)

In [18]:
actions_test.shape

(100, 3)

In [19]:
# tf.squared_difference?

In [34]:
# metric_mse = tf.reduce_mean(tf.square(y_plh - y))
metric_mse = tf.reduce_mean(tf.squared_difference(y_plh, y))

In [35]:
# metric_mse = tf.metrics.mean_squared_error(labels=y_plh, predictions=y)

In [36]:
casted = tf.cast(metric_mse, tf.float32)

In [37]:
casted.eval(feed_dict={x_plh: obs_test, y_plh: actions_test})

11.423635

In [30]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [31]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [32]:
accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels})

0.89230001